# Nosana AI Inference Tutorial with Python

## Introduction

Nosana is a GPU marketplace that makes AI compute accessible and affordable. This tutorial demonstrates how to interact with deployed AI models on the Nosana network, including multimodal understanding and text-to-image generation.

## What You'll Learn

- Connect to deployed Nosana AI services via API
- Perform multimodal image understanding tasks
- Generate images from text prompts
- Deploy your own AI services on Nosana
- Cost optimization for GPU-powered AI inference

## Prerequisites

- Python 3.8+
- Basic understanding of AI/ML concepts
- For deployment: Node.js, SOL tokens (0.05 SOL minimum), NOS tokens

## Setup and Installation

In [ ]:
# Uncomment to install required packages
# !pip install gradio_client requests pillow matplotlib python-dotenv -q

In [ ]:
import os
import json
import requests
import base64
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from gradio_client import Client
from typing import Dict, List, Optional

## Connecting to Deployed Nosana AI Services

This example uses a live multimodal AI service deployed on Nosana that provides:
- **Multimodal Understanding**: Analyze images and answer questions
- **Text-to-Image Generation**: Create images from text descriptions

In [ ]:
class NosanaAIClient:
    """Python client for interacting with Nosana-deployed AI services"""
    
    def __init__(self, service_url: str):
        self.service_url = service_url.rstrip('/')
        self.client = Client(service_url)
        print(f"🚀 Connected to Nosana AI service: {service_url}")
    
    def understand_image(self, 
                        image_path: str, 
                        question: str,
                        seed: int = 42,
                        top_p: float = 0.95,
                        temperature: float = 0.1) -> str:
        """Analyze image and answer questions using multimodal AI"""
        
        try:
            print(f"🔍 Analyzing image: {image_path}")
            print(f"❓ Question: {question}")
            
            result = self.client.predict(
                image_path,
                question,
                seed,
                top_p,
                temperature,
                fn_index=2
            )
            
            print(f"✅ Analysis complete!")
            return result
            
        except Exception as e:
            error_msg = f"❌ Image analysis failed: {str(e)}"
            print(error_msg)
            return error_msg
    
    def generate_image(self,
                      prompt: str,
                      seed: int = 12345,
                      cfg_weight: float = 5.0,
                      temperature: float = 1.0) -> str:
        """Generate images from text prompts"""
        
        try:
            print(f"🎨 Generating image from prompt: {prompt[:50]}...")
            
            result = self.client.predict(
                prompt,
                seed,
                cfg_weight,
                temperature,
                fn_index=3
            )
            
            print(f"✅ Image generation complete!")
            return result
            
        except Exception as e:
            error_msg = f"❌ Image generation failed: {str(e)}"
            print(error_msg)
            return error_msg
    
    def test_connection(self) -> bool:
        """Test if the service is accessible"""
        try:
            response = requests.get(self.service_url, timeout=10)
            if response.status_code == 200:
                print(f"✅ Service is online and accessible")
                return True
            else:
                print(f"⚠️ Service returned status: {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ Connection test failed: {e}")
            return False

# Initialize client with deployed Nosana service
NOSANA_SERVICE_URL = "https://53mnfyl8sretv69hx9dhhnaqaxj8qrzwg6qpfd62tbtq.node.k8s.prd.nos.ci/"
ai_client = NosanaAIClient(NOSANA_SERVICE_URL)

# Test connection
ai_client.test_connection()

## Multimodal Image Understanding

In [ ]:
# Example 1: Analyze a meme (as shown in the document)
meme_url = "https://raw.githubusercontent.com/gradio-app/gradio/main/test/test_files/bus.png"

# Download and display the image
response = requests.get(meme_url)
img = Image.open(BytesIO(response.content))
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis('off')
plt.title('Image for Analysis')
plt.show()

# Analyze the image
analysis = ai_client.understand_image(
    image_path=meme_url,
    question="What do you see in this image? Describe it in detail.",
    seed=42,
    top_p=0.95,
    temperature=0.1
)

print("\n📝 AI Analysis:")
print("=" * 50)
print(analysis)
print("=" * 50)

In [ ]:
# Example 2: Multiple questions about the same image
questions = [
    "What objects can you identify in this image?",
    "What colors are prominent in this image?",
    "Is this image taken indoors or outdoors?",
    "What might be the context or setting of this image?"
]

print("🧠 Multi-Question Analysis:")
print("=" * 60)

for i, question in enumerate(questions, 1):
    print(f"\n{i}. {question}")
    answer = ai_client.understand_image(
        image_path=meme_url,
        question=question,
        temperature=0.2  # Lower temperature for more consistent answers
    )
    print(f"💡 Answer: {answer}")
    print("-" * 40)

## Text-to-Image Generation

In [ ]:
# Example prompts for image generation
creative_prompts = [
    "A cute and adorable baby fox with big brown eyes, autumn leaves in the background",
    "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k",
    "A glass of red wine on a reflective surface, cinematic lighting",
    "Master shifu raccoon wearing drip attire as a street gangster"
]

def generate_and_display_images(prompts: List[str], max_images: int = 2):
    """Generate multiple images and display results"""
    
    results = []
    
    for i, prompt in enumerate(prompts[:max_images]):
        print(f"\n🎨 Generating image {i+1}/{min(len(prompts), max_images)}...")
        
        # Generate image with different seeds for variety
        result = ai_client.generate_image(
            prompt=prompt,
            seed=12345 + i * 100,  # Different seed for each image
            cfg_weight=5.0,
            temperature=1.0
        )
        
        results.append({
            'prompt': prompt,
            'result': result
        })
        
        print(f"✅ Image {i+1} generated successfully!")
        print(f"📁 Result path: {result}")
    
    return results

# Generate images
print("🚀 Starting image generation with Nosana AI...")
generated_images = generate_and_display_images(creative_prompts, max_images=2)

# Display results
print("\n📊 Generation Summary:")
for i, img_data in enumerate(generated_images, 1):
    print(f"{i}. Prompt: {img_data['prompt'][:50]}...")
    print(f"   Result: {img_data['result']}")

## Advanced AI Workflows

In [ ]:
class NosanaAIWorkflow:
    """Advanced workflows combining multiple AI capabilities"""
    
    def __init__(self, ai_client: NosanaAIClient):
        self.client = ai_client
    
    def analyze_and_enhance_prompt(self, image_url: str) -> str:
        """Analyze an image and create an enhanced prompt for generation"""
        
        # First, analyze the image
        analysis = self.client.understand_image(
            image_path=image_url,
            question="Describe this image in detail, focusing on style, colors, composition, and artistic elements that could be used to generate a similar image.",
            temperature=0.3
        )
        
        # Create enhanced prompt based on analysis
        enhanced_prompt = f"High quality artistic image inspired by: {analysis}, detailed, professional photography, 8k resolution"
        
        return enhanced_prompt
    
    def batch_image_analysis(self, image_urls: List[str], base_question: str) -> Dict:
        """Analyze multiple images with the same question"""
        
        results = {}
        
        for i, url in enumerate(image_urls):
            print(f"📸 Analyzing image {i+1}/{len(image_urls)}...")
            
            analysis = self.client.understand_image(
                image_path=url,
                question=base_question,
                seed=42 + i,
                temperature=0.2
            )
            
            results[f"image_{i+1}"] = {
                "url": url,
                "analysis": analysis
            }
        
        return results
    
    def progressive_image_generation(self, base_prompt: str, variations: int = 3) -> List[str]:
        """Generate variations of an image with progressive modifications"""
        
        style_modifiers = [
            "photorealistic, highly detailed",
            "artistic, painterly style, vibrant colors", 
            "minimalist, clean composition, soft lighting"
        ]
        
        results = []
        
        for i in range(min(variations, len(style_modifiers))):
            modified_prompt = f"{base_prompt}, {style_modifiers[i]}"
            
            print(f"🎨 Generating variation {i+1}: {style_modifiers[i]}")
            
            result = self.client.generate_image(
                prompt=modified_prompt,
                seed=12345 + i * 50,
                cfg_weight=5.0 + i * 0.5,  # Slightly different CFG for each
                temperature=1.0
            )
            
            results.append(result)
        
        return results

# Initialize workflow manager
workflow = NosanaAIWorkflow(ai_client)

# Example: Progressive image generation
base_prompt = "A serene mountain landscape at sunset"
print(f"🌄 Creating variations of: {base_prompt}")

variations = workflow.progressive_image_generation(base_prompt, variations=2)

print("\n✅ Generated variations:")
for i, result in enumerate(variations, 1):
    print(f"{i}. {result}")

## Deploying Your Own AI Services on Nosana

In [ ]:
class NosanaDeploymentManager:
    """Manage Nosana AI service deployments"""
    
    def __init__(self):
        self.markets = {
            "nvidia-3060": "7AtiXMSH6R1jjBxrcYjehCkkSF7zvYWte63gwEDBcGHq",
            "nvidia-3090": "97G9NnvBDQ2WpKu6fasoMsAKmfj63C9rhysJnkeWodAf"
        }
    
    def create_gradio_ai_job(self, 
                            model_name: str = "multimodal-ai",
                            port: int = 7860) -> Dict:
        """Create job definition for Gradio-based AI service"""
        
        job_definition = {
            "version": "0.1",
            "type": "container",
            "meta": {"trigger": "python-ai-deploy"},
            "ops": [{
                "type": "container/run",
                "id": f"{model_name}-service",
                "args": {
                    "cmd": [
                        "python", "-c",
                        "import gradio as gr; "
                        "import torch; "
                        "print('GPU Available:', torch.cuda.is_available()); "
                        "gr.Interface(lambda x: f'Hello from Nosana GPU! Input: {x}', 'text', 'text').launch(server_name='0.0.0.0', server_port=7860)"
                    ],
                    "expose": port,
                    "image": "pytorch/pytorch:2.0.1-cuda11.7-cudnn8-devel",
                    "gpu": True,
                    "env": {
                        "CUDA_VISIBLE_DEVICES": "0",
                        "GRADIO_SERVER_NAME": "0.0.0.0"
                    }
                }
            }]
        }
        
        return job_definition
    
    def save_job_definition(self, job_def: Dict, filename: str = "ai_service.json") -> str:
        """Save job definition to file"""
        with open(filename, 'w') as f:
            json.dump(job_def, f, indent=2)
        return filename
    
    def deploy_ai_service(self, 
                         job_file: str,
                         market: str = "nvidia-3060",
                         timeout: int = 120) -> Dict:
        """Deploy AI service to Nosana network"""
        
        import subprocess
        
        cmd = [
            "nosana", "job", "post",
            "--file", job_file,
            "--market", market,
            "--timeout", str(timeout)
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            return {"success": True, "output": result.stdout}
        except subprocess.CalledProcessError as e:
            return {"success": False, "error": e.stderr}

# Example: Create and save deployment definition
deployment_manager = NosanaDeploymentManager()

# Create AI service job definition
ai_job = deployment_manager.create_gradio_ai_job("custom-ai-service")
job_file = deployment_manager.save_job_definition(ai_job, "custom_ai_service.json")

print(f"✅ AI service job definition created: {job_file}")
print("\n📋 Job Definition Preview:")
print(json.dumps(ai_job, indent=2))

print("\n💡 To deploy this service, ensure you have:")
print("   • Nosana CLI installed and configured")
print("   • Sufficient SOL (0.05 minimum) and NOS tokens")
print(f"   • Run: nosana job post --file {job_file} --market nvidia-3060 --timeout 120")

## Cost Analysis for AI Inference

In [ ]:
class NosanaCostAnalyzer:
    """Analyze costs for AI inference on Nosana"""
    
    def __init__(self):
        self.market_rates = {
            "nvidia-3060": 0.000043,  # NOS per second
            "nvidia-3090": 0.000115   # NOS per second  
        }
        
        # Estimated processing times for different AI tasks
        self.task_times = {
            "image_analysis": 10,      # seconds per image
            "image_generation": 30,    # seconds per image
            "batch_processing": 5,     # seconds per item in batch
            "model_loading": 60        # initial model loading time
        }
    
    def calculate_inference_cost(self, 
                               market: str,
                               task_type: str, 
                               num_requests: int,
                               include_startup: bool = True) -> Dict:
        """Calculate cost for AI inference tasks"""
        
        if market not in self.market_rates:
            return {"error": f"Unknown market: {market}"}
        
        if task_type not in self.task_times:
            return {"error": f"Unknown task type: {task_type}"}
        
        rate_per_second = self.market_rates[market]
        task_time = self.task_times[task_type]
        
        # Calculate total time
        processing_time = task_time * num_requests
        startup_time = self.task_times["model_loading"] if include_startup else 0
        total_time = processing_time + startup_time
        
        # Calculate costs
        total_cost = rate_per_second * total_time
        cost_per_request = total_cost / num_requests if num_requests > 0 else 0
        
        return {
            "market": market,
            "task_type": task_type,
            "num_requests": num_requests,
            "processing_time_seconds": processing_time,
            "startup_time_seconds": startup_time,
            "total_time_seconds": total_time,
            "total_cost_nos": total_cost,
            "cost_per_request_nos": cost_per_request,
            "hourly_rate_nos": rate_per_second * 3600
        }
    
    def compare_markets_for_workload(self, 
                                   task_type: str, 
                                   num_requests: int):
        """Compare costs across markets for specific workload"""
        
        print(f"💰 Cost Comparison: {num_requests} {task_type} requests")
        print("=" * 60)
        
        for market in self.market_rates:
            cost_info = self.calculate_inference_cost(market, task_type, num_requests)
            
            print(f"\n🏪 {market}:")
            print(f"   Total Cost: {cost_info['total_cost_nos']:.6f} NOS")
            print(f"   Per Request: {cost_info['cost_per_request_nos']:.6f} NOS") 
            print(f"   Total Time: {cost_info['total_time_seconds']} seconds")
            print(f"   Hourly Rate: {cost_info['hourly_rate_nos']:.4f} NOS/hour")
    
    def recommend_market(self, 
                        task_type: str, 
                        num_requests: int, 
                        budget_nos: float) -> str:
        """Recommend best market based on budget and workload"""
        
        recommendations = []
        
        for market in self.market_rates:
            cost_info = self.calculate_inference_cost(market, task_type, num_requests)
            
            if cost_info['total_cost_nos'] <= budget_nos:
                recommendations.append({
                    'market': market,
                    'cost': cost_info['total_cost_nos'],
                    'efficiency': num_requests / cost_info['total_cost_nos']
                })
        
        if not recommendations:
            return "Budget too low for any market"
        
        # Sort by efficiency (requests per NOS)
        best = max(recommendations, key=lambda x: x['efficiency'])
        return best['market']

# Cost analysis examples
cost_analyzer = NosanaCostAnalyzer()

# Compare costs for different AI tasks
tasks = [
    ("image_analysis", 100),
    ("image_generation", 50),
    ("batch_processing", 500)
]

for task_type, num_requests in tasks:
    cost_analyzer.compare_markets_for_workload(task_type, num_requests)
    
    # Get recommendation for modest budget
    recommended = cost_analyzer.recommend_market(task_type, num_requests, budget_nos=1.0)
    print(f"\n🎯 Recommended market for {task_type}: {recommended}")
    print("="*80)

## Best Practices & Next Steps

### AI Inference Optimization
- **Batch requests** when possible to reduce startup costs
- **Choose appropriate GPU markets** based on model complexity
- **Monitor processing times** to optimize timeout settings
- **Cache results** for repeated queries

### Cost Management
- Start with nvidia-3060 for testing and development
- Use nvidia-3090 for production and complex models
- Factor in model loading time for cost calculations
- Set reasonable timeouts to avoid unnecessary charges

### Security & Reliability
- Never hardcode API keys or private keys
- Implement proper error handling and retries
- Monitor service availability and performance
- Use environment variables for sensitive configuration

## Summary

You've learned how to:
✅ Connect to deployed Nosana AI services via Python
✅ Perform multimodal image understanding and text-to-image generation
✅ Build advanced AI workflows with multiple capabilities
✅ Deploy your own AI services on Nosana's GPU network
✅ Analyze and optimize costs for AI inference workloads

**Next Steps:**
1. Experiment with the live Nosana AI service using your own images and prompts
2. Deploy your own AI models using the deployment templates
3. Build production workflows combining multiple AI capabilities
4. Join the [Nosana Discord](https://discord.gg/nosana) community
5. Explore [dashboard.nosana.com](https://dashboard.nosana.com) for advanced management

The Nosana network makes powerful GPU-accelerated AI accessible to everyone at fraction of traditional cloud costs!